## Muestreo

Para analizar la calidad del conjunto de datos, seleccionar variables, limpiar y transformar los datos y finalmente determinar un número $k$ de clusters partiremos de un muestreo del conjunto de datos inicial, con un tamaño de muestra del 3% con respecto al original. 

Para el muestreo utilizamos el método de __reservoir sapling__ visto en clase.

Como el conjunto de datos cuenta con más de 5 millones de registros, segun su [documentación](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3) fijamos el tamaño de muestra:




In [ ]:
n_muestras <- 5.4*(1*10**6)*0.3
t1 <- Sys.time()
dir()
print(n_muestras)
set.seed(0)

In [ ]:
path.file <- "Fire_Department_Calls_for_Service.csv" #--path donde se encuentra el conjunto de datos original 
n_filas_read <- 500000 # la rm nos permite cargar este numero de registros y agilizar el muestreo

connection <- file(path.file, open = "r")
#--nombre de las columnas (primera fila)
#col_names  <- read.csv(connection, nrows = 1, header = TRUE)
#--definimos nuestro buffer (muestra) y los rellenamos con las primeras n_muestras filas
buffer <- read.csv(connection, nrows = n_muestras, header = TRUE, stringsAsFactors = FALSE)

#--indice que nos permitira generar los numeros aleatorios correctamente
posicion_inicial <- n_muestras
random_unif <- function(x) sample.int(x,1)

contador <- 1
repeat{
  print(paste0("Posicion inicial: ", posicion_inicial))
  
   #--leemos una parte del archivo
  temp <- read.csv(connection, nrows = n_filas_read, header = FALSE)

  #--indices que controlan el maximo de cada numero aleatorio
  maximo <- c(1:nrow(temp)) + posicion_inicial

  #--generamos numeros aleatorios de forma vectorizada, segun esto solo permuta los indices
  j = vapply(maximo, random_unif, FUN.VALUE = integer(1))

  #--observamos cuales de los numeros aleatorios son menores que nuestra muestra
  idx <- j <= n_muestras

  #--sustituimos los que resultaron menores
  buffer[j[idx], ] <- temp[idx, ]
  
  print(paste0("iteracion: ", contador))
  contador <- contador + 1
  #--redefinimos la posicion inicial para la siguiente iteracion
  posicion_inicial <- posicion_inicial + nrow(temp)
  
  #--si el numero de filas leidas es menor que el esperado,
  #-asumimos que se acabo el archivo y salimos del ciclo
  if(nrow(temp) < n_filas_read)
    break
}
t2 <- Sys.time()
print(t2 - t1) #(Time difference of 9.29019 mins mins )
#--guardar nuestra muestra para un futuro analisis
write.csv(buffer, paste0("muestra_", path.file), row.names = FALSE )
close(connection)

In [ ]:
library(readr)
data <- read_csv('muestra3_Fire_Department_Calls_for_Service.csv')

In [ ]:
print(dim(data))

# Selección de variables 


Después de leer la documentación y entender la estructura de la data, decidimos que el cluster que realizaremos tendrá como objetivo encontrar grupos de llamadas parecidas entre sí y contrastaremos estos grupos con la etiqueta que poseen los datos en la columna `Call Type Group`.
Después de revisar la documentación  descartamos las columnas `Call.Type`, `RowID`, `Unit ID`, `Incident Number`, `Unit Type`, `Unit sequence in call dispatch` al igual que `Location`. Las primeras por no aportar información extra y la última porque la documentación no proporciona el tipo de proyección utilizado para referencias las coordenadas de los puntos.

In [ ]:
data$RowID <- data$Unit.ID <- data$Incident.Number <- data$Location <- data$Unit.Type <- data$Unit.sequence.in.call.dispatch <- data$Call.Type <- NULL

In [ ]:
names(data)

En vista de que las siguientes columnas no se encuentran [documentadas en el diccionario de datos correspondiente](https://data.sfgov.org/api/views/nuek-vuh3/files/ddb7f3a9-0160-4f07-bb1e-2af744909294?download=true&filename=FIR-0002_DataDictionary_fire-calls-for-service.xlsx) procedemos a eliminarlas:
`Current Police Districts`, `Neighborhoods - Analysis Boundaries` , `Zip Codes`, `Neighborhoods (old)`, `Police Districts`, `Civic Center Harm Reduction Project Boundary`, `HSOC Zones` y  `Central Market/Tenderloin Boundary Polygon - Updated`

In [ ]:
data[ , c('Current.Police.Districts', 'Neighborhoods...Analysis.Boundaries', 'Zip.Codes', 'Neighborhoods..old.', 'Police.Districts', 
          'Civic.Center.Harm.Reduction.Project.Boundary', 'HSOC.Zones', 'Central.Market.Tenderloin.Boundary.Polygon...Updated')] <- NULL

In [ ]:
head(data)

# Limpieza de datos 


Procedemos a retener el último registro de cada llamada, el cual contiene la información acumulada de los anteriores. 

In [ ]:
library(dplyr)
library(lubridate)
data$Received.DtTm <- mdy_hms(data$Received.DtTm)
data %>% group_by(Call.Number ) %>% arrange(Call.Number , Received.DtTm ) %>% mutate( flag1 = n() ,flag2 = row_number()) -> data
data %>% filter(flag1 ==flag2 ) -> data
data$flag1 <- data$flag2 <- NULL
data$Available.DtTm <- NULL
data$Call.Date <- mdy(data$Call.Date)
data$Watch.Date <- mdy(data$Watch.Date)
data$Entry.DtTm <- mdy_hms(data$Entry.DtTm)
data$Dispatch.DtTm <- mdy_hms(data$Dispatch.DtTm)
data$Response.DtTm <- mdy_hms( data$Response.DtTm) 
data$On.Scene.DtTm <- mdy_hms( data$On.Scene.DtTm) 
data$Transport.DtTm <- mdy_hms(data$Transport.DtTm) 
data$Hospital.DtTm <- mdy_hms(data$Hospital.DtTm) 


Como suponemos que la duración de la llamada está correlacionada con su clasificación con las variables de tipo fecha (`Call.Date`, `Watch.Date`, `Received.DtTm`, `Entry.DtTm`, `Dispatch.DtTm`, `Response.DtTm`, `On.Scene.DtTm`, `Transport.DtTm`, `Hospital.DtTm`) obtenemos la duración aproximada de la llamada. 


In [ ]:
head(data)

In [ ]:
#install.packages('reshape2')
library(reshape2)

In [ ]:
data.t <- melt(data, id = c('Call.Number','Call.Final.Disposition', 'Address', 'City', 'Zipcode.of.Incident', 'Battalion', 
                            'Station.Area', 'Box', 'Original.Priority', 'Priority', 'Final.Priority', 'ALS.Unit', 'Call.Type.Group', 
                            'Number.of.Alarms',  'Fire.Prevention.District', 'Supervisor.District', 
                            'Neighborhooods...Analysis.Boundaries', 'Supervisor.Districts', 'Fire.Prevention.Districts' ) ) %>%
          filter( variable %in% c('Received.DtTm', 'Entry.DtTm', 'Dispatch.DtTm', 'Response.DtTm', 'On.Scene.DtTm', 'Transport.DtTm', 'Hospital.DtTm' )) 
head(data.t)

In [ ]:
data.t %>% group_by (Call.Number, Call.Final.Disposition, Address, City, Zipcode.of.Incident, Battalion, 
                            Station.Area, Box, Original.Priority, Priority, Final.Priority, ALS.Unit, Call.Type.Group, 
                            Number.of.Alarms,  Fire.Prevention.District, Supervisor.District, 
                            Neighborhooods...Analysis.Boundaries, Supervisor.Districts, Fire.Prevention.Districts  ) %>% 
      summarise( min.t = min(value, na.rm=TRUE), max.t =max(value, na.rm=TRUE)) -> data.t
data.t <- data.t %>% mutate(Call.seconds = max.t - min.t)
data.t$min.t <- data.t$max.t <- NULL 

In [ ]:
print(dim(data.t))
tail(data.t)

# K-means on-line

Como aplicación principal de machine learning tenemos dos objetivos, el primero es encontrar grupos de llamadas similares entre sí y que estos se contrastarlos con la etiqueta de 4 tipos que la data tiene, el segundo es asignar un tipo de llamada a los que no la tienen razón que desconocemos pues la documentación del dataset no lo declara. 


In [ ]:
alpha <- 0.1
data <- as.data.frame(data.t)
kmeans.online.b.init <- function(data, k, alpha){
  # clousure para distribuir la eleccion del elemento k
  data <- data
  alpha <- alpha
  function(k){
    # Entradas 
    # data (data.frame): Dataframe donde las observaciones son los elementos a clusterizar y las columnas son las variables
    # k (int): Numero de cluster requerido
    # alpha (numeric): learning rate
    # Salida
    # kmeans.online con los elementos:
    # tabla.master (data.frame): Dos columnas, la primera con el id de la observacion y la segunda con el label del cluster
    # statas.intra (vector): Vector con la media d ela varianza intraelementos
    tabla.master <- data.frame(Obs = row.names(data), Cluster= rep(-Inf, dim(data)[1]))
    # inicializacion alatoria entre el minimo y maximo de cada variable
    stats.min <- sapply(data, min, na.rm=TRUE)
    stats.max <- sapply(data, max, na.rm=TRUE)
    set.seed(0)
    centroides <- mapply(function(x, y) {runif(k, x, y)},  stats.min, stats.max) 
    # termina inicializacion de centroides
    
    # comienza kmeans proceso online
    
    for( i in 1:dim(data)[1])
    {
      #i <- 11
      #print(i)
      # comienza asignacion de cluster mas cercano
      observacion.en.juego <- as.matrix(data[i, ])
      m.temp <- as.matrix(rbind(observacion.en.juego, centroides))
      distancias <- dist(m.temp)
      m.distancias <- as.matrix(distancias)
      k.i <- which.min(m.distancias[1, 2:(k+1)])
      tabla.master$Cluster[i] <- k.i
      # termina asignacion de cluster mas cercano 
      # update de cluster
      centroides[k.i, ] <- centroides[k.i, ] + alpha*observacion.en.juego
    }
    stats <- rep(-Inf, k)
    for ( i in 1:k)
    {
      index <- which( tabla.master$Cluster == i)
      data.subset <- data[ index, ]
      stats.i <- dist(data.subset)
      stats[i] <- sum(stats.i) # asumimos independencia entre las variables
    }
    kmeans.online <- list( tabla.master =tabla.master, statas.intra = stats)
    return(kmeans.online)
  }
}

In [ ]:
Y <- data.frame(y=data$Call.Type.Group)
row.names(Y) <- row.names(data) <- data$Call.Number
data$Call.Type.Group <- data$Call.Number <- NULL
# hacemos un cambio de encoding de las variables ordinales para no hacer una matriz con muchas variables ---IMPORTANTE

In [ ]:
index <- which(is.na(Y$y))
Y$y <- as.character(Y$y) 
Y$y[index] <- 'No asignado'

In [ ]:
index <- which(sapply(data, class) == 'character')
normalizar <- function(x, na.rm = FALSE) (x - mean(x, na.rm = na.rm)) / sd(x, na.rm)

In [ ]:
# mas preprocesamiento de datos
for (i in index)
    {
    index.na <- which( is.na(data[, i] ))
    data[index.na, i] <- 'NULL'
    temp <- factor( data[, i])
    data[, i] <- normalizar(as.numeric( temp))
}

In [ ]:
sum(table(Y$y)) # distribucion del etiquetamiento del tipo de llamada
print(table( Y$y))

In [ ]:
kmeans.online.b <- kmeans.online.b.init(data = data, alpha = alpha)

In [ ]:
set.seed(0)
cluster <- 1:4
t1 <- Sys.time()
for( i in cluster)
{
  print(i)
  res <- kmeans.online.b(k=2*i+1)
  cluster[i] <- sum(res$statas.intra)
  print(cluster)
}
t2 <- Sys.time()

In [ ]:
t2-t1

In [ ]:
2*1:4 + 1

In [ ]:
plot((cluster),type = 'l', xlab = '2*x + 1 ', ylab = 'Var intra cluster')

In [ ]:
plot(abs(diff(cluster)),type = 'l',  xlab = '2*x + 1 ', ylab = 'diff(Var) intra cluster')

Por lo que consideramos que una elección de $k=7$ es adecuada y ejecutamos el algoritmo de kmeans online implementado sobre el conjunto completo de datos. 

In [1]:
k = 7

In [2]:
library(reshape2)
library(dplyr)
library(lubridate)
data <- read.csv('Fire_Department_Calls_for_Service.csv', stringsAsFactors = FALSE)
print(head(data))

data$RowID <- data$Unit.ID <- data$Incident.Number <- data$Location <- data$Unit.Type <- data$Unit.sequence.in.call.dispatch <- data$Call.Type <- NULL
data[ , c('Current.Police.Districts', 'Neighborhoods...Analysis.Boundaries', 'Zip.Codes', 'Neighborhoods..old.', 'Police.Districts', 
          'Civic.Center.Harm.Reduction.Project.Boundary', 'HSOC.Zones', 'Central.Market.Tenderloin.Boundary.Polygon...Updated')] <- NULL
data$Received.DtTm <- mdy_hms(data$Received.DtTm)
data %>% group_by(Call.Number ) %>% arrange(Call.Number , Received.DtTm ) %>% mutate( flag1 = n() ,flag2 = row_number()) -> data
data %>% filter(flag1 ==flag2 ) -> data
data$flag1 <- data$flag2 <- NULL
data$Available.DtTm <- NULL
data$Call.Date <- mdy(data$Call.Date)
data$Watch.Date <- mdy(data$Watch.Date)
data$Entry.DtTm <- mdy_hms(data$Entry.DtTm)
data$Dispatch.DtTm <- mdy_hms(data$Dispatch.DtTm)
data$Response.DtTm <- mdy_hms( data$Response.DtTm) 
data$On.Scene.DtTm <- mdy_hms( data$On.Scene.DtTm) 
data$Transport.DtTm <- mdy_hms(data$Transport.DtTm) 
data$Hospital.DtTm <- mdy_hms(data$Hospital.DtTm) 
data.t <- melt(data, id = c('Call.Number','Call.Final.Disposition', 'Address', 'City', 'Zipcode.of.Incident', 'Battalion', 
                            'Station.Area', 'Box', 'Original.Priority', 'Priority', 'Final.Priority', 'ALS.Unit', 'Call.Type.Group', 
                            'Number.of.Alarms',  'Fire.Prevention.District', 'Supervisor.District', 
                            'Neighborhooods...Analysis.Boundaries', 'Supervisor.Districts', 'Fire.Prevention.Districts' ) ) %>%
          filter( variable %in% c('Received.DtTm', 'Entry.DtTm', 'Dispatch.DtTm', 'Response.DtTm', 'On.Scene.DtTm', 'Transport.DtTm', 'Hospital.DtTm' )) 
data.t %>% group_by (Call.Number, Call.Final.Disposition, Address, City, Zipcode.of.Incident, Battalion, 
                            Station.Area, Box, Original.Priority, Priority, Final.Priority, ALS.Unit, Call.Type.Group, 
                            Number.of.Alarms,  Fire.Prevention.District, Supervisor.District, 
                            Neighborhooods...Analysis.Boundaries, Supervisor.Districts, Fire.Prevention.Districts  ) %>% 
      summarise( min.t = min(value, na.rm=TRUE), max.t =max(value, na.rm=TRUE)) -> data.t
data.t <- data.t %>% mutate(Call.seconds = max.t - min.t)
data.t$min.t <- data.t$max.t <- NULL 
data.t <- as.data.frame(data.t)

print(dim(data.t))
Y <- data.frame(y=data.t$Call.Type.Group)
row.names(Y) <- row.names(data.t) <- data.t$Call.Number
data.t$Call.Type.Group <- data.t$Call.Number <- NULL
# hacemos un cambio de encoding de las variables ordinales para no hacer una matriz con muchas variables ---IMPORTANTE
index <- which(is.na(Y$y))
Y$y <- as.character(Y$y) 
Y$y[index] <- 'No asignado'
index <- which(sapply(data.t, class) == 'character')

normalizar <- function(x, na.rm = FALSE) (x - mean(x, na.rm = na.rm)) / sd(x, na.rm)
# mas preprocesamiento de datos
for (i in index)
    {
    index.na <- which( is.na(data.t[, i] ))
    data.t[index.na, i] <- 'NULL'
    temp <- factor( data.t[, i])
    data.t[, i] <- normalizar(as.numeric( temp))
}
head(data.t)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




[1] 5369711      44


,Call.Number,Unit.ID,Incident.Number,Call.Type,Call.Date,Watch.Date,Received.DtTm,Entry.DtTm,Dispatch.DtTm,Response.DtTm,⋯,Supervisor.Districts,Fire.Prevention.Districts,Current.Police.Districts,Neighborhoods...Analysis.Boundaries,Zip.Codes,Neighborhoods..old.,Police.Districts,Civic.Center.Harm.Reduction.Project.Boundary,HSOC.Zones,Central.Market.Tenderloin.Boundary.Polygon...Updated
,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,201560006,86,20064818,Medical Incident,06/04/2020,06/03/2020,06/04/2020 12:00:17 AM,06/04/2020 12:02:00 AM,06/04/2020 12:02:09 AM,06/04/2020 12:02:13 AM,⋯,9,NA,2,37,62,37,2,NA,NA,NA
2,201560006,E48,20064818,Medical Incident,06/04/2020,06/03/2020,06/04/2020 12:00:17 AM,06/04/2020 12:02:00 AM,06/04/2020 12:02:09 AM,06/04/2020 12:04:08 AM,⋯,9,NA,2,37,62,37,2,NA,NA,NA
3,201560006,RA48,20064818,Medical Incident,06/04/2020,06/03/2020,06/04/2020 12:00:17 AM,06/04/2020 12:02:00 AM,06/04/2020 12:02:09 AM,06/04/2020 12:04:12 AM,⋯,9,NA,2,37,62,37,2,NA,NA,NA
4,201560012,94,20064819,Medical Incident,06/04/2020,06/03/2020,06/04/2020 12:03:15 AM,06/04/2020 12:05:31 AM,06/04/2020 12:05:48 AM,06/04/2020 12:06:04 AM,⋯,7,2,4,2,28859,2,7,NA,NA,NA
5,201560019,E01,20064820,Outside Fire,06/04/2020,06/03/2020,06/04/2020 12:09:59 AM,06/04/2020 12:11:01 AM,06/04/2020 12:11:53 AM,06/04/2020 12:13:50 AM,⋯,9,12,2,8,28855,6,2,NA,NA,NA
6,201560029,53,20064821,Medical Incident,06/04/2020,06/03/2020,06/04/2020 12:17:34 AM,06/04/2020 12:18:32 AM,06/04/2020 12:18:54 AM,06/04/2020 12:19:02 AM,⋯,11,7,6,39,28852,41,9,1,1,NA
7,201560029,E05,20064821,Medical Incident,06/04/2020,06/03/2020,06/04/2020 12:17:34 AM,06/04/2020 12:18:32 AM,06/04/2020 12:18:54 AM,,⋯,11,7,6,39,28852,41,9,1,1,NA
8,201560029,QRV1,20064821,Medical Incident,06/04/2020,06/03/2020,06/04/2020 12:17:34 AM,06/04/2020 12:18:32 AM,06/04/2020 12:19:48 AM,06/04/2020 12:19:48 AM,⋯,11,7,6,39,28852,41,9,1,1,NA
9,201560029,RC1,20064821,Medical Incident,06/04/2020,06/03/2020,06/04/2020 12:17:34 AM,06/04/2020 12:18:32 AM,06/04/2020 12:18:54 AM,06/04/2020 12:21:14 AM,⋯,11,7,6,39,28852,41,9,1,1,NA


In [8]:
alpha <- 0.1
t1 <- Sys.time()
tabla.master <- data.frame(Obs = row.names(data.t), Cluster= rep(-Inf, dim(data.t)[1]))
    # inicializacion alatoria entre el minimo y maximo de cada variable
head(tabla.master)

In [10]:
stats.min <- sapply(data.t, min, na.rm=TRUE)
stats.max <- sapply(data.t, max, na.rm=TRUE)
set.seed(0)
centroides <- mapply(function(x, y) {runif(k, x, y)},  stats.min, stats.max) 
    # termina inicializacion de centroides

# comienza kmeans proceso online
    for( i in 1:dim(data.t)[1])
    {
      if(i %% 100000 ==0 )
      print(i)
      # comienza asignacion de cluster mas cercano
      observacion.en.juego <- as.matrix(data.t[i, ])
      m.temp <- as.matrix(rbind(observacion.en.juego, centroides))
      distancias <- dist(m.temp)
      m.distancias <- as.matrix(distancias)
      k.i <- which.min(m.distancias[1, 2:(k+1)])
      tabla.master$Cluster[i] <- k.i
      # termina asignacion de cluster mas cercano 
      # update de cluster
      centroides[k.i, ] <- centroides[k.i, ] + alpha*observacion.en.juego
    }
t2 <- Sys.time()

[1] 10000
[1] 20000
[1] 30000
[1] 40000
[1] 50000
[1] 60000
[1] 70000
[1] 80000
[1] 90000
[1] 100000
[1] 110000
[1] 120000
[1] 130000
[1] 140000
[1] 150000
[1] 160000
[1] 170000
[1] 180000
[1] 190000
[1] 200000
[1] 210000
[1] 220000
[1] 230000
[1] 240000
[1] 250000
[1] 260000
[1] 270000
[1] 280000
[1] 290000
[1] 300000
[1] 310000
[1] 320000
[1] 330000
[1] 340000
[1] 350000
[1] 360000
[1] 370000
[1] 380000
[1] 390000
[1] 400000
[1] 410000
[1] 420000
[1] 430000
[1] 440000
[1] 450000
[1] 460000
[1] 470000
[1] 480000
[1] 490000
[1] 500000
[1] 510000
[1] 520000
[1] 530000
[1] 540000
[1] 550000
[1] 560000
[1] 570000
[1] 580000
[1] 590000
[1] 600000
[1] 610000
[1] 620000
[1] 630000
[1] 640000
[1] 650000
[1] 660000
[1] 670000
[1] 680000
[1] 690000
[1] 700000
[1] 710000
[1] 720000
[1] 730000
[1] 740000
[1] 750000
[1] 760000
[1] 770000
[1] 780000
[1] 790000
[1] 800000
[1] 810000
[1] 820000
[1] 830000
[1] 840000
[1] 850000
[1] 860000
[1] 870000
[1] 880000
[1] 890000
[1] 900000
[1] 910000
[1] 9200

In [14]:
t2 - t1

Time difference of 4.378615 hours

In [30]:
index <- which(Y$y == '')
Y$y <- as.character(Y$y) 
Y$y[index] <- 'No asignado'

In [33]:
tabla.master$etiqueta.original <- Y$y

In [34]:
table(tabla.master$etiqueta.original, tabla.master$Cluster)

                              
                                    1      2      3      4      5      6      7
  Alarm                          4856  89670  88468   5102   4916   4952  25291
  Fire                            792  11599  11633    848    819    757   3804
  No asignado                   97676 374520 310971  93878  95194 118890 160748
  Non Life-threatening          38009  94026  93398  39577  39552  38906  58144
  Potentially Life-Threatening  36267 158199 157760  37490  37684  36297  72185

In [36]:
write.csv(tabla.master, file='resultados_cluster_7.csv')

# Conclusión 

Con la clasificación no supervisada resultado de kmeans online con 7 grupos, tenemos las siguientes observaciones:
   1. En relación a nuestra primera hipótesis sobre encontrar grupos de llamadas similares podemos establecer que efectivamente un $k=7$ es apropiado. En el notebook `JessVega-Copy2`repetimos el mismo ejercicio pero con una configuración diferente:
       a) En este set up el número de clusters se fijó a 5 un número que surge de manera intuitiva al considerar las 4 categorías que se encuentran en la data más la que está ausente (valor ullo). En este experimento el resultado está sumamente sesgado donde la mayoría de las llamadas son clasificadas al grupo 5. 

2. Sobre la segunda hipótesis podemos concluir con base en la tabla de frecuencias anterior que lo más verosímil es asignar a la etiqueta de los datos `Potentially Life-Threatening` el label 2, a la etiqueta de los datos `Non Life-threatening` el label 4, a la etiqueta de los datos `Fire` el label 7, a la etiqueta de los datos `Alarm` el label 3 y finalmente a todas las llamadas a las que no se les asignó una llamada podemos atribuirlas principalmente al grupo 2.   

Hay dos comentarios importantes finales. 
Si bien el número 7 de cluster pareciera ser grande su existencia se respalda porque las 5 etiquetas presentes en la data se distribuyen transversalmente en ellos. 
Por último la distribución de la etiqueta `No asignado` proporciona ideas de un futuro análisis para proponer una nueva clasificación con más de 7 niveles. 
